In [5]:
%matplotlib inline

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from itertools import combinations
from mlxtend import frequent_patterns

In [23]:
df = pd.read_csv("monkey.csv")
df = df.drop(['TID'], axis=1)
df.columns

Index([' List'], dtype='object')

In [27]:
def convertToOnehot(df, col):
    return df.join(df[col].str.get_dummies(", ")).drop([' List'], axis=1)

In [29]:
def checkForCombinations(combList, combTuple, r):
    isPresent = True
    for comb in combinations(combTuple, r):
        if comb not in combList:
            isPresent = False
            break
    return isPresent

In [31]:
def combineCols(df, col1, col2):
    return (df[col1] & df[col2])

In [33]:
def getCount(df, col):
    return len(df.loc[df[col] == 1])

In [43]:
def getAllConfidence(elements, countDict, minConfidence):
    for element in elements:
        for comb in combinations(element, len(element) - 1):
            comb = list(comb)
            remaining = list(set(element) - set(comb))
            if getConfidence(comb, remaining, countDict) > minConfidence:
                print(comb,"->",remaining, getConfidence(comb, remaining, countDict))
            if len(element) > 2 and (getConfidence(remaining, comb, countDict) > minConfidence):
                print(remaining,"->",comb, getConfidence(remaining, comb, countDict))

In [38]:
def getConfidence(items_given, support_items, countDict):
    items_given.sort()
    support_items = support_items + items_given
    support_items.sort()
    items_given_str = "_".join(items_given)
    item_support_str = "_".join(support_items)
    item_support = item_support_str
    items = list(countDict.keys())
    if (items_given_str not in items) or (item_support not in items):
        return 0
    else:
        return (countDict[item_support]/ countDict[items_given_str])

In [14]:
def apriori(df, col, s_count):
    df = pd.DataFrame(df[col])
    df = convertToOnehot(df, col)
    items = df.columns
    countDf = df.sum()
    countDict = countDf.loc[countDf >= s_count].to_dict()
    print(countDict)
    items = list(countDict.keys())
    combHist = []
    combPrev = list(combinations(items, 1))
    combNext = []
    for i in range(2, len(items) - 1):
        for comb in combinations(items, i):
            if checkForCombinations(combPrev, comb, (i-1)):
                combProp = "_".join(str(c) for c in comb)
                col1 = "_".join(str(c) for c in comb[:-1])
                col2 = str(comb[-1])
                df[combProp] = combineCols(df, col1, col2)
                countTemp = getCount(df, combProp)
                if countTemp >= s_count:
                    combNext.append(comb)
                    countDict[combProp] = getCount(df, combProp)
        combHist.append(combPrev)
        combPrev = combNext
    return countDict

In [40]:
countDict2 = apriori(df,' List', 3)

{'E': 4, 'K': 4, 'M': 3, 'O': 3, 'Y': 3}


In [41]:
elements = [s.split("_") for s in countDict2.keys() if len(s) > 2]

In [45]:
getAllConfidence(elements, countDict2, 0.7)

['E'] -> ['K'] 1.0
['K'] -> ['E'] 1.0
['E'] -> ['O'] 0.75
['O'] -> ['E'] 1.0
['K'] -> ['O'] 0.75
['O'] -> ['K'] 1.0
['E', 'K'] -> ['O'] 0.75
['O'] -> ['E', 'K'] 1.0
['E', 'O'] -> ['K'] 1.0
['K'] -> ['E', 'O'] 0.75
['K', 'O'] -> ['E'] 1.0
['E'] -> ['K', 'O'] 0.75


In [46]:
countDict2

{'E': 4,
 'K': 4,
 'M': 3,
 'O': 3,
 'Y': 3,
 'E_K': 4,
 'E_O': 3,
 'K_O': 3,
 'E_K_O': 3}